In [ ]:
!pip install -U transformers datasets peft accelerate bitsandbytes trl huggingface_hub

In [ ]:
from huggingface_hub import notebook_login

# Hugging Face 로그인
notebook_login()

In [3]:
import torch
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

In [5]:
model_id = 'Qwen/Qwen2.5-3B-Instruct'
output_dir = "./qwen2.5-3b-customer-anlysis"

In [6]:
# 4-bit 양자화된 모델 로드를 위한 설정
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',             # 양자화 타입 지정
    bnb_4bit_use_double_quant=True,        # 이중 양자화(양자화된 가중치를 한번 더 양자화)
    bnb_4bit_compute_dtype=torch.bfloat16  # 16bit의 부동소수점 데이터 타입 사용
)

# 모델 및 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quant_config,
    device_map="auto",
    trust_remote_code=True
)

# 모델 학습 준비
model = prepare_model_for_kbit_training(model)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [7]:
peft_config = LoraConfig(
    r=16, 
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"], # Qwen의 레이어 이름
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, peft_config)

In [ ]:
SYSTEM_PROMPT = """
상담 스크립트에서 고객의 성향을 분류하세요.
전체적인 맥락을 참고하되 판단의 근거는 고객의 발화에 한정합니다.

### 분류 규칙
1. 제시된 성향 키워드 중 가장 적절한 한가지만 선택
2. 오직 키워드만 추출 (예: N5)

### 성향 키워드 목록
- N1 (실용주의형): 불필요한 말 없이 목적 달성에만 집중함
- N2 (친화적수다형): 사적인 이야기나 본인 상황을 길게 설명함
- N3 (신중/보안 중시형): 신중하고 의심을 보임
- S1 (급한성격형): 빠른 결론과 처리를 선호함
- S2 (꼼꼼상세형): 상세한 설명을 요구함
- S3 (감정호소형): 본인의 사정을 감정적으로 호소하며 공감을 구함
- S4 (이해부족형): 설명을 잘 이해하지 못함
- S5 (디지털네이티브): 앱 사용에 능숙하며 전문 용어나 신조어 사용에 거부감이 없음
- S6 (반복민원형): 과거의 상담 이력을 언급하며 해결되지 않은 불만을 반복 제기함
- S7 (불만항의형): 분노, 짜증을 드러내며 항의함
"""

In [9]:
df = pd.read_csv("train.csv")
dataset = Dataset.from_pandas(df)

In [10]:
dataset

Dataset({
    features: ['source_id', 'script', 'output'],
    num_rows: 2
})

In [11]:
dataset = dataset.rename_columns({col: col.strip() for col in dataset.column_names})

def tokenize_function(examples):
    texts = []
    for script, output in zip(examples['script'], examples['output']):
        text = (
            f"<|im_start|>system\n{SYSTEM_PROMPT}<|im_end|>\n"
            f"<|im_start|>user\n{script}<|im_end|>\n"
            f"<|im_start|>assistant\n{output}<|im_end|>"
        )
        texts.append(text)
    
    tokenized = tokenizer(
        texts,
        truncation=True,
        max_length=1024, # 2048이 너무 크면 1024로 줄여서 OOM 방지
        padding="max_length", 
        return_tensors=None, # 리스트 형태로 반환해야 map 함수가 작동함
    )
    
    tokenized["labels"] = [list(ids) for ids in tokenized["input_ids"]]
    return tokenized

# 데이터셋 변환 (컬럼 삭제 필수)
tokenized_dataset = dataset.map(
    tokenize_function, 
    batched=True, 
    remove_columns=dataset.column_names
)

# 학습 설정
# 패딩이 완료되었으므로 가장 기본적인 Collator를 사용하거나 생략 가능합니다.
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=2, # 메모리 부족 시 1로 변경
    gradient_accumulation_steps=8,
    learning_rate=1e-4,
    num_train_epochs=3,
    logging_steps=10,
    save_strategy="epoch",
    fp16=True,
    optim="paged_adamw_32bit",
    report_to="none",
    remove_unused_columns=False # labels를 유지하기 위해 False 설정
)

# 4. Trainer 실행
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

trainer.train()

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an except

TrainOutput(global_step=3, training_loss=2.1991360982259116, metrics={'train_runtime': 10.8979, 'train_samples_per_second': 0.551, 'train_steps_per_second': 0.275, 'total_flos': 103392730939392.0, 'train_loss': 2.1991360982259116, 'epoch': 3.0})

In [12]:
# 모델 저장
trainer.model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"학습 완료. 모델이 {output_dir}에 저장되었습니다.")

학습 완료. 모델이 ./qwen2.5-3b-customer-anlysis에 저장되었습니다.


In [13]:
# 로컬에 저장된 모델과 토크나이저 업로드
repo_id = 'ansui/qwen2.5-3b-customer-anlysis'

trainer.model.push_to_hub(repo_id)
tokenizer.push_to_hub(repo_id)

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

README.md: 0.00B [00:00, ?B/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

CommitInfo(commit_url='https://huggingface.co/ansui/qwen2.5-3b-customer-anlysis/commit/2d5e5ec84fc0447cc3c5f6c22eb2e8aa2b9581d5', commit_message='Upload tokenizer', commit_description='', oid='2d5e5ec84fc0447cc3c5f6c22eb2e8aa2b9581d5', pr_url=None, repo_url=RepoUrl('https://huggingface.co/ansui/qwen2.5-3b-customer-anlysis', endpoint='https://huggingface.co', repo_type='model', repo_id='ansui/qwen2.5-3b-customer-anlysis'), pr_revision=None, pr_num=None)

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

base_model_id = "Qwen/Qwen2.5-3B-Instruct"
adapter_model_id = "ansui/qwen2.5-3b-customer-anlysis"

tokenizer = AutoTokenizer.from_pretrained(adapter_model_id)

# 베이스 모델 로드
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

# 베이스 모델에 LoRA 어댑터 입히기
model = PeftModel.from_pretrained(base_model, adapter_model_id)
model.eval()

# 테스트 함수 정의
def analyze_customer(script):
    prompt = (
        f"<|im_start|>system\n{SYSTEM_PROMPT}"
        f"<|im_start|>user\n{script}<|im_end|>\n"
        f"<|im_start|>assistant\n"
    )
    
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    
    with torch.no_state():
        outputs = model.generate(
            **inputs, 
            max_new_tokens=10, 
            temperature=0.1,
            eos_token_id=tokenizer.eos_token_id
        )
    
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result.split("assistant\n")[-1].strip()

# 실제 테스트
test_script = """
상담사: 무엇을 도와드릴까요?
손님: 아니 카드 결제 문자가 왜 두 번이나 와요? 빨리 확인해서 취소해 줘요!
"""
print(f"분류 결과: {analyze_customer(test_script)}")

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/496 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]